In [1]:
using CSV
using DataFrames
using Revise
using StatsBase
using StochasticPrograms
using Gurobi
using JuMP
cd("/Users/frankiecho/Library/CloudStorage/OneDrive-TheUniversityofQueensland/Documents/GitHub/koala-uncertainty/")
#includet("code/optim-functions.jl")

Import data files

In [2]:
cost_df = CSV.read("data/meanWTA_10yr.csv", DataFrame);
habitat_df = CSV.read("data/habitat_suitw_graham.csv", DataFrame);

Load dataframes as matrices
- N: number of properties
- T: number of timesteps
- S: number of scenarios

In [3]:
cost = Matrix(cost_df[:, [:cost2025, :cost2035, :cost2045, :cost2055, :cost2065, :cost2075, :cost2085]]);
N = size(cost,1);
T = size(cost,2);
S = length(unique(habitat_df.climate_model));

In [4]:
M = zeros(N, T, S);
for (s, cm) in enumerate(unique(habitat_df.climate_model))
    df = filter(row -> row.climate_model == cm, habitat_df);
    select!(df, Not([:climate_model, :NewPropID]))
    M[:,:,s] = Matrix(df);
end

# Solve a two-stage stochastic optimisation model
Written as:

$$ \min_{x,y,w} \sum_i \sum_{t<t'} c_{it}x_i + \sum_s p_s \sum_i \sum_{t\geq t'} ((\beta + c_{it}) y_{is} + (\gamma - c_{it})w_{is})$$
$$ s.t. \sum_i m_{its} x_i \geq K \quad \forall t < t', s \in S$$
$$ \sum_i m_{its} (x_i + y_{is} - w_{is}) \geq K \quad \forall t \geq t', s \in S$$
$$ x_i + y_{is} \leq 1 \quad \forall i \in N, s \in S $$
$$ x_i \geq w_{is} \quad \forall i \in N, s \in S $$
$$ x, y, w \in [0,1] $$

where
* $i$: properties (out of $N$)
* $t$: timesteps - $t'$ is the timestep when uncertainty about $m$ is revealed
* $s$: scenarios (out of $S$)
* $x$: properties with covenant signed before $t'$
* $y$: properties with covenant newly added in the second stage (after $t'$)
* $w$: properties with covenant terminated at timestep $t'$
* $c$: cost (mean WTA)
* $m$: quality metric (suitability-weighed koala habitat)
* $K$: objective for the amount of suitability-weighted koala habitats that must be achieved
* $\beta$: additional costs incurred for adding covenants in the second stage
* $\gamma$: additional costs incurred for terminating covenants in the second stage
* $p$: probability of each scenario

Solve four versions:
1. No recourse allowed: all decisions made in the first time (i.e. $y = 0$ & $w = 0$)
2. Only adding new units in year 2055 (i.e. $w = 0$)
3. Only terminating covenants in year 2055 (i.e. $y = 0$)
4. Full recourse: add and terminating units allowed

In [5]:
cost

17276×7 Matrix{Float64}:
 14816.6        18061.3        22016.6        …  39880.1        48613.6
 18039.4        21989.9        26805.5           48554.5        59187.7
     1.36672e5      1.66602e5      2.03087e5         3.67864e5      4.48425e5
 13903.1        16947.7        20659.2           37421.3        45616.3
 13740.4        16749.4        20417.5           36983.4        45082.6
  7201.65        8778.77       10701.3        …  19383.9        23628.8
 26742.3        32598.7        39737.6           71979.2        87742.2
  6878.47        8384.82       10221.0           18514.0        22568.5
  7179.83        8752.17       10668.8           19325.1        23557.2
 16966.5        20682.0        25211.3           45666.7        55667.5
     ⋮                                        ⋱      ⋮          
 42208.5        51451.9        62719.6               1.13608e5      1.38487e5
 10964.9        13366.1        16293.2           29512.9        35976.1
 30203.4        36817.8        448

In [29]:
@stochastic_model two_stage_model begin
  @stage 1 begin
    @parameters begin
      K = 70.0
      tt = 4
      β = 0
      γ = 0
      M = M
    end
    @decision(two_stage_model, 0 <= x[i in 1:N] <= 1)
    @objective(two_stage_model, Min, sum(cost[i,t]*x[i] for i in 1:N, t in 1:T))
    for t=1:(tt-1)
      for s=1:S
        @constraint(two_stage_model, sum(M[i, t, s] * x[i] for i in 1:N) >= K)
      end
    end
  end
  @stage 2 begin
    @uncertain m[i in 1:N, t in 1:T]
    @recourse(two_stage_model, 0 <= y[i in 1:N] <= 1)
    @recourse(two_stage_model, 0 <= w[i in 1:N] <= x[i in 1:N])
    @objective(two_stage_model, Min, sum((β + cost[i,t]) * y[i] + (γ - cost[i,t]) * w[i] for i in 1:N, t in tt:T))
    for t=tt:T
      @constraint(two_stage_model, sum(m[i,t] * (x[i] + y[i] - w[i]) for i in 1:N) >= K)
    end
    for i=1:N
      @constraint(two_stage_model, x[i] + y[i] <= 1)
    end
  end
end


Two-Stage Stochastic Model

minimize f₀(x) + 𝔼[f(x,ξ)]
  x∈𝒳

where

f(x,ξ) = min  f(y; x, ξ)
              y ∈ 𝒴 (x, ξ)


In [33]:
ξ1 = @scenario m[i in 1:N, t in 1:T]=M[:,:,1] probability = 1/2
ξ2 = @scenario m[i in 1:N, t in 1:T]=M[:,:,2] probability = 1/2

Scenario with probability 0.5 and underlying data:

[0.06821642533047058 0.07109529182755195 … 0.07043271564453757 0.07152350022485145; 0.1137932185847261 0.11859551484976207 … 0.11749025792582428 0.11930981806218725; … ; 0.002426390332838917 0.002333163053785501 … 0.0023130880769657984 0.0022693740158057727; 0.0 0.0 … 0.0 0.0]

In [34]:
sp = instantiate(two_stage_model, [ξ1, ξ2], optimizer = Gurobi.Optimizer)

Set parameter Username
Academic license - for non-commercial use only - expires 2024-01-08


ArgumentError: ArgumentError: invalid index: true of type Bool

In [10]:
solution_no_recourse    = fcn_two_stage_opt(cost, M, 70.0, ones(S)/S, 4; add_recourse = false, terminate_recourse = false)
solution_add_only       = fcn_two_stage_opt(cost, M, 70.0, ones(S)/S, 4; add_recourse = true, terminate_recourse = false)
solution_terminate_only = fcn_two_stage_opt(cost, M, 70.0, ones(S)/S, 4; add_recourse = false, terminate_recourse = true)
solution_full_recourse  = fcn_two_stage_opt(cost, M, 70.0, ones(S)/S, 4; add_recourse = true , terminate_recourse = true)

Set parameter Username
Academic license - for non-commercial use only - expires 2023-12-16
Set parameter Username
Academic license - for non-commercial use only - expires 2023-12-16
Set parameter Username
Academic license - for non-commercial use only - expires 2023-12-16
Set parameter Username
Academic license - for non-commercial use only - expires 2023-12-16


(model = A JuMP Model
Minimization problem with:
Variables: 1261148
Objective function type: AffExpr
`AffExpr`-in-`MathOptInterface.GreaterThan{Float64}`: 622188 constraints
`AffExpr`-in-`MathOptInterface.LessThan{Float64}`: 621936 constraints
`VariableRef`-in-`MathOptInterface.GreaterThan{Float64}`: 1261148 constraints
`VariableRef`-in-`MathOptInterface.LessThan{Float64}`: 1261148 constraints
Model mode: AUTOMATIC
CachingOptimizer state: ATTACHED_OPTIMIZER
Solver name: Gurobi
Names registered in the model: w, x, y, obj_value = 5.1489790658401385e7, x = [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], y = [0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0], w = [0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0])

Plot bar chart showing the objective values

In [11]:
obj_values = [solution_no_recourse.obj_value; solution_add_only.obj_value; solution_terminate_only.obj_value; solution_full_recourse.obj_value];
solution_list = ["no_recourse", "add_only", "terminate_only", "full_recourse"];
obj_value_df = DataFrame(hcat(solution_list, obj_values), ["Solution", "Objective Value"]);
CSV.write("data/obj_values.csv", obj_value_df);


In [ ]:
function fcn_tidy_two_stage_solution(solution, prop_id, climate_models)
    out = hcat(prop_id, solution.x, solution.y, solution.w)
    y_array = ["y_" * c for c in climate_models];
    w_array = ["w_" * c for c in climate_models];
    colnames = vcat(["NewPropID"; "x"], y_array, w_array);
    return DataFrame(out, colnames);
end

function fcn_tidy_two_stage_solution_sum(solution, prop_id)
    out = hcat(prop_id, solution.x, sum(solution.y, dims=2), sum(solution.w, dims=2))
    colnames = vcat(["NewPropID"; "x"; "sum_y"; "sum_w"]);
    return DataFrame(out, colnames);
end

climate_models = unique(habitat_df.climate_model)
solution_df = map((c->fcn_tidy_two_stage_solution_sum(c, cost_df.NewPropID)), [solution_no_recourse, solution_add_only, solution_terminate_only, solution_full_recourse]);

In [ ]:
CSV.write("data/solution_no_recourse.csv", solution_df[1]);
CSV.write("data/solution_add_only.csv", solution_df[2]);
CSV.write("data/solution_terminate_only.csv", solution_df[3]);
CSV.write("data/solution_full_recourse.csv", solution_df[4]);
